# EDA de datos de precipitaciones La Plata

Se realiza un analisis exploratorio de los datos correspondientes a la estación La Plata AERO, provistos por el Servicio Meteorologico Nacional. Lo que se realizará es una verificación de datos faltantes, calidad de los datos y finalmente el calculo de los indices de precipitación extrema

In [ ]:
!pip install pandas
!pip install pyhomogeneity

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pyhomogeneity import pettitt_test, snht_test, buishand_u_test


In [ ]:
from google.colab import drive
PATH = "/content/drive/MyDrive/TP-Mate4/Datos/"
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/TP-Mate4/Datos/La-Plata-Aero", skiprows=3, names=["fecha", "pp", "ruido 1", "ruido 2"])

Primero defino unas funciones que serán de utilidad para la realización de los análisis.

In [ ]:
def df_to_precip_series(df, fecha_col="fecha", precip_col="pp"):
    df = df.copy()
    serie = pd.Series(df[precip_col].values, index=df[fecha_col])
    serie = serie.sort_index()

    return serie

def break_point_plot(result, mu1, mu2, mn, mx, loc):

    plt.figure(figsize=(16,6))
    plt.plot(serie_pp, label="Observation")
    plt.hlines(mu1, xmin=mn, xmax=loc, linestyles='--', colors='orange',lw=1.5, label='mu1 : ' + str(round(mu1,2)))
    plt.hlines(mu2, xmin=loc, xmax=mx, linestyles='--', colors='g', lw=1.5, label='mu2 : ' + str(round(mu2,2)))
    plt.axvline(x=loc, linestyle='-.' , color='red', lw=1.5, label='Change point : '+ loc.strftime('%Y-%m-%d') + '\n p-value : ' + str(result.p))

    plt.title('Precipitaciones diarias')
    plt.xlabel('Fecha')
    plt.ylabel('Precipitación (mm)')
    plt.legend(loc='upper right')

def calcular_indices_etccdi(
    serie,
    base_start='1961-01-01',
    base_end='1990-12-31',
    freq='YE',
    periodos=None,
    por_estacion=False,
    modo_periodos=False
):

    serie = serie.sort_index().fillna(0.0)

    base_period = serie.loc[base_start:base_end]
    p95 = base_period[base_period > 1.0].quantile(0.95)
    p99 = base_period[base_period > 1.0].quantile(0.99)

    if periodos is None:
        periodos = [(serie.index.min(), serie.index.max())]

    estaciones = {
        "Verano_DJF": [12, 1, 2],
        "Otoño_MAM": [3, 4, 5],
        "Invierno_JJA": [6, 7, 8],
        "Primavera_SON": [9, 10, 11]
    }

    resultados = []

    for inicio, fin in periodos:
        serie_periodo = serie.loc[inicio:fin]

        if por_estacion:
            for nombre_estacion, meses in estaciones.items():
                serie_estacion = serie_periodo[serie_periodo.index.month.isin(meses)]
                if len(serie_estacion) == 0:
                    continue

                if modo_periodos:
                    indices = _calcular_indices_agregado(serie_estacion, p95, p99)
                    indices['Periodo'] = f"{pd.to_datetime(inicio).year}-{pd.to_datetime(fin).year}"
                    indices['Estacion'] = nombre_estacion
                    resultados.append(indices)
                else:
                    indices = _calcular_indices_por_año(serie_estacion, freq, p95, p99)
                    indices['Estacion'] = nombre_estacion
                    resultados.append(indices)
        else:
            if modo_periodos:
                indices = _calcular_indices_agregado(serie_periodo, p95, p99)
                indices['Periodo'] = f"{pd.to_datetime(inicio).year}-{pd.to_datetime(fin).year}"
                indices['Estacion'] = "Anual"
                resultados.append(indices)
            else:
                indices = _calcular_indices_por_año(serie_periodo, freq, p95, p99)
                indices['Estacion'] = "Anual"
                resultados.append(indices)

    return pd.concat(resultados).reset_index(drop=True)

def _calcular_indices_por_año(serie, freq, p95, p99):
    """
    Calcula índices ETCCDI para cada año.
    """
    grupos = serie.resample(freq)

    df = pd.DataFrame({
        "PRCPTOT": grupos.sum(),
        "SDII": grupos.apply(lambda x: x[x > 1].sum() / max((x > 1).sum(), 1)),
        "CWD": grupos.apply(lambda x: _max_consecutive(x, cond=lambda v: v >= 1)),
        "CDD": grupos.apply(lambda x: _max_consecutive(x, cond=lambda v: v < 1)),
        "R10mm": grupos.apply(lambda x: (x >= 10).sum()),
        "R20mm": grupos.apply(lambda x: (x >= 20).sum()),
        "R40mm": grupos.apply(lambda x: (x >= 40).sum()),
        "R95pTOT": grupos.apply(lambda x: x[x > p95].sum()),
        "R99pTOT": grupos.apply(lambda x: x[x > p99].sum()),
        "Rx1day": grupos.apply(lambda x: x.max()),
        "Rx5day": grupos.apply(lambda x: x.rolling(5).sum().max())
    })

    df['Año'] = df.index.year
    return df.reset_index(drop=True)


def _calcular_indices_agregado(serie, p95, p99):
    """
    Calcula índices ETCCDI agregados para todo el periodo.
    """
    return pd.DataFrame({
        "PRCPTOT": [serie.sum()],
        "SDII": [serie[serie > 1].sum() / max((serie > 1).sum(), 1)],
        "CWD": [_max_consecutive(serie, cond=lambda v: v >= 1)],
        "CDD": [_max_consecutive(serie, cond=lambda v: v < 1)],
        "R10mm": [(serie >= 10).sum()],
        "R20mm": [(serie >= 20).sum()],
        "R40mm": [(serie >= 40).sum()],
        "R95pTOT": [serie[serie > p95].sum()],
        "R99pTOT": [serie[serie > p99].sum()],
        "Rx1day": [serie.max()],
        "Rx5day": [serie.rolling(5).sum().max()],
    })


def _max_consecutive(serie, cond):
    """
    Calcula la máxima racha consecutiva que cumple una condición.
    """
    max_racha = 0
    racha = 0
    for val in serie:
        if cond(val):
            racha += 1
            max_racha = max(max_racha, racha)
        else:
            racha = 0
    return max_racha


In [ ]:
df = df[["fecha", "pp"]]
print(df.head())
print(df.info())

       fecha pp
0 1961-01-01  0
1 1961-01-02  0
2 1961-01-03  0
3 1961-01-04  0
4 1961-01-05  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23557 entries, 0 to 23556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   23557 non-null  datetime64[ns]
 1   pp      23557 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 368.2+ KB
None


A modo de facilitar el computo, casteo los datos de fecha y precipitación a sus tipos correspondientes, además de crear nuevas columnas donde se aislan los datos de año, mes y dia.

In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"])
df["pp"] = pd.to_numeric(df["pp"], errors="coerce")

df["año"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month
df["dia"] = df["fecha"].dt.day

print(df.head())
print(df.info())

       fecha   pp   año  mes  dia
0 1961-01-01  0.0  1961    1    1
1 1961-01-02  0.0  1961    1    2
2 1961-01-03  0.0  1961    1    3
3 1961-01-04  0.0  1961    1    4
4 1961-01-05  0.0  1961    1    5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23557 entries, 0 to 23556
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   23557 non-null  datetime64[ns]
 1   pp      23486 non-null  float64       
 2   año     23557 non-null  int32         
 3   mes     23557 non-null  int32         
 4   dia     23557 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(3)
memory usage: 644.3 KB
None


Algo importante a verificar es la presencia de datos faltantes, para decidir luego que hacer con ellos. Lo que se hará entonces es lo siguiente:

1. Obtener los datos faltantes y el porcentaje de los mismos con respecto al total de datos presentes.
2. Una vez obtenidos los datos faltantes, verificar como se distribuyen los mismos. ¿Los datos faltantes son consecutivos o más bien aislados?

In [ ]:
faltantes = df.isna().sum()
faltantes_pct = faltantes * 100 / len(df)

print("Valores faltantes por columna:")
print(faltantes)
print("\nPorcentaje de faltantes:")
print(faltantes_pct)

Valores faltantes por columna:
fecha     0
pp       71
año       0
mes       0
dia       0
dtype: int64

Porcentaje de faltantes:
fecha    0.000000
pp       0.301397
año      0.000000
mes      0.000000
dia      0.000000
dtype: float64


Se puede observar que los datos faltantes son 71 en total, lo que representa tan solo el 0.3% de los datos totales.

In [ ]:
faltantes = df[df["pp"].isna()].copy()
faltantes = faltantes.sort_values("fecha")
faltantes["consecutivo"] = (faltantes["fecha"].diff().dt.days == 1)
faltantes["grupo"] = (~faltantes["consecutivo"]).cumsum()

rachas = (
    faltantes.groupby("grupo")
    .agg(inicio=("fecha","min"), fin=("fecha","max"), dias=("fecha","count"))
    .reset_index(drop=True)
)

print("Rachas de días consecutivos con datos faltantes:")
print(rachas)

Rachas de días consecutivos con datos faltantes:
       inicio        fin  dias
0  1969-05-10 1969-05-10     1
1  1969-12-21 1969-12-21     1
2  1972-01-01 1972-01-31    31
3  1981-11-14 1981-11-14     1
4  1983-03-01 1983-03-31    31
5  1987-01-26 1987-01-26     1
6  1990-09-10 1990-09-10     1
7  1993-08-07 1993-08-07     1
8  1994-04-29 1994-04-29     1
9  2018-12-14 2018-12-14     1
10 2020-01-19 2020-01-19     1


Algo importante a observar aquí es que si bien son pocos los datos faltantes, en su mayoria estos se distribuyen en dos meses en particular: Enero de 1972 y Marzo de 1983. Esto es algo a tener en cuenta en los análisis posteriores, ya que si se calculan máximos anuales o

In [ ]:
df = df.dropna(subset=["pp"])
df = df[df['año'] != 2025]
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 23305 entries, 0 to 23375
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   23305 non-null  datetime64[ns]
 1   pp      23305 non-null  float64       
 2   año     23305 non-null  int32         
 3   mes     23305 non-null  int32         
 4   dia     23305 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(3)
memory usage: 819.3 KB
None


Ahora realizamos test de homogeneidad sobre la serie de precipitaciones. utilizamos las tres más comunes para estos casos: Petit, SHNT y Buishand.

In [ ]:
pettitt_res = pettitt_test(serie_pp, alpha=0.05)
print(pettitt_res)

snht_res = snht_test(serie_pp, sim=10000)
print(snht_res)

buishand_res = buishand_u_test(serie_pp)
print(buishand_res)

Pettitt_Test(h=np.False_, cp=np.str_('2006-03-17'), p=np.float64(0.3388), U=np.float64(1920652.0), avg=mean(mu1=np.float64(2.839500091224229), mu2=np.float64(2.7252987467210725)))
SNHT_Test(h=np.False_, cp=np.str_('1976-09-27'), p=np.float64(0.2283), T=np.float64(8.269129695001707), avg=mean(mu1=np.float64(2.4998600419874037), mu2=np.float64(2.9053215077605326)))
Buishand_U_Test(h=np.False_, cp=np.str_('1976-09-27'), p=np.float64(0.1381), U=np.float64(0.29339080096660625), avg=mean(mu1=np.float64(2.4998600419874037), mu2=np.float64(2.9053215077605326)))


Podemos observar que en general si bien sugieren un posible candidato, estadisticamente hablando no hay alguno que sea significativo. Esto lo podemos ver con los p-value que se obtiene de cada test, que son mayores al definido (0.5). Por lo tanto, no hay evidencia suficiente para rechazar la homogeneidad.

# Cálculo de indices

Indices de precipitación extrema confeccionados por el ETCCDI:

| Nº | Índice    | Nombre                                | Definición / Cálculo | Unidad |
|----|-----------|---------------------------------------------------|----------------------|--------|
| 17 | RX1day    | Máxima precipitación en 1 día                      | Sea RRᵢⱼ la precipitación diaria en el día *i* del período *j*. Se define: **RX1dayⱼ = máx(RRᵢⱼ)**. | mm |
| 18 | RX5day    | Máxima precipitación en 5 días consecutivos        | Sea RRₖⱼ la precipitación acumulada en el intervalo de 5 días *k* del período *j* (k se identifica por el último día). Se define: **RX5dayⱼ = máx(RRₖⱼ)**. | mm |
| 19 | SDII      | Índice de intensidad diaria simple                 | Sea RRwⱼ la precipitación diaria en los días húmedos (*RR ≥ 1 mm*) del período *j* y W el número de esos días. Se define: **SDIIⱼ = Σ(RRwⱼ) / W**. | mm/día |
| 20 | R10mm     | Días con precipitación ≥ 10 mm                     | Número de días en el período *j* con **RRᵢⱼ ≥ 10 mm**. | días |
| 21 | R20mm     | Días con precipitación ≥ 20 mm                     | Número de días en el período *j* con **RRᵢⱼ ≥ 20 mm**. | días |
| 22 | Rnnmm     | Días con precipitación ≥ umbral definido por el usuario | Número de días en el período *j* con **RRᵢⱼ ≥ nn mm**, donde *nn* es un umbral elegido por el usuario. | días |
| 23 | CDD       | Máxima racha de días secos consecutivos            | Longitud máxima de la secuencia de días consecutivos en el período *j* con **RRᵢⱼ < 1 mm**. | días |
| 24 | CWD       | Máxima racha de días húmedos consecutivos          | Longitud máxima de la secuencia de días consecutivos en el período *j* con **RRᵢⱼ ≥ 1 mm**. | días |
| 25 | R95pTOT   | Precipitación debida a días muy húmedos (> percentil 95) | Sea RRwⱼ la precipitación diaria en los días húmedos del período *j* y RRwₙ⁹⁵ el percentil 95 de la precipitación en días húmedos del período base *n* (p.ej., 1961–1990). Se define: **R95pTOTⱼ = Σ(RRwⱼ) para RRwⱼ > RRwₙ⁹⁵**. | mm |
| 26 | R99pTOT   | Precipitación debida a días extremadamente húmedos (> percentil 99) | Igual que el anterior, pero usando **RRwₙ⁹⁹**, el percentil 99 del período base: **R99pTOTⱼ = Σ(RRwⱼ) para RRwⱼ > RRwₙ⁹⁹**. | mm |
| 27 | PRCPTOT   | Precipitación total en días húmedos (≥ 1 mm)        | Sea RRwⱼ la precipitación diaria en los días húmedos (*RR ≥ 1 mm*) del período *j*. Se define: **PRCPTOTⱼ = Σ(RRwⱼ)**. | mm |

Lo que se hace a continuación es calcular los indices mencionados anteriormente siguiendo los siguientes casos:

- 1er caso: Serie anual de indices comprendiendo todo el periodo de datos
- 2do caso: Serie anual de indices por estaciones del año comprendiendo todo el periodo de datos
- 3er caso: Serie anual de indices separado en dos periodos (1961-1990; 1991-2020)
- 4to caso: Serie anual de indices por estaciones del año separado en dos periodos (1961-1990; 1991-2020)


In [ ]:
indices_anuales = calcular_indices_etccdi(
    serie=serie_pp,
    periodos=None,
    por_estacion=False,
)

print(indices_anuales)


    PRCPTOT       SDII  CWD  CDD  R10mm  R20mm  R40mm  R95pTOT  R99pTOT  \
0     820.8  12.926984    3   33     26     13      5    170.5      0.0   
1     765.0  16.465217    5   43     24     14      2    131.0    131.0   
2    1533.8  19.336709    5   15     44     30      8    507.6    204.4   
3     888.0  15.471930    4   25     28     11      3    292.5    237.8   
4     593.3  10.408929    3   21     23      9      0      0.0      0.0   
..      ...        ...  ...  ...    ...    ...    ...      ...      ...   
59    805.4  14.753704    3   29     24     15      6     99.0      0.0   
60    750.5  14.188462    4   25     25     14      1     53.0      0.0   
61    579.9  10.200000    8   36     22      8      1      0.0      0.0   
62    903.6  15.418966    4   38     22     14      6    386.2     88.5   
63   1163.9  16.746377    4   24     34     15      8    405.5    293.0   

    Rx1day  Rx5day   Año Estacion  
0     59.0    93.0  1961    Anual  
1    131.0   150.4  1962   

In [ ]:
indices_estaciones = calcular_indices_etccdi(
    serie=serie_pp,
    periodos=None,
    por_estacion=True,
    freq='YE'
)

print(indices_estaciones)


     PRCPTOT       SDII  CWD  CDD  R10mm  R20mm  R40mm  R95pTOT  R99pTOT  \
0      233.5  12.278947    2   10      5      4      1     59.0      0.0   
1      150.4   9.986667    2   16      5      3      0      0.0      0.0   
2      441.3  23.115789    3   10     13      7      3    165.2    108.2   
3      204.7  25.387500    1   25      3      1      1    148.6    148.6   
4      192.3  11.925000    2   15      8      3      0      0.0      0.0   
..       ...        ...  ...  ...    ...    ...    ...      ...      ...   
251    249.1  19.015385    3   29      7      5      3     99.0      0.0   
252    212.1  13.143750    4   20      8      3      0      0.0      0.0   
253     98.8   8.881818    2   26      3      1      1      0.0      0.0   
254    253.1  13.905556    2   21      8      4      2    110.1      0.0   
255    180.3  11.162500    3   18      7      2      1      0.0      0.0   

     Rx1day  Rx5day   Año       Estacion  
0      59.0    93.0  1961     Verano_DJF  
1

In [ ]:
periodos = [('1961-01-01', '1990-12-31'),
            ('1991-01-01', '2020-12-31')]

indices_anuales_por_periodo = calcular_indices_etccdi(
    serie_pp,
    periodos=periodos,
    por_estacion=False,
    modo_periodos=True
)

print(indices_anuales_por_periodo)

   PRCPTOT       SDII  CWD  CDD  R10mm  R20mm  R40mm  R95pTOT  R99pTOT  \
0  29836.9  14.560818   10   55    944    473    148   6962.2   2209.4   
1  32156.1  15.228394    7   41   1003    552    183   7876.3   1983.0   

   Rx1day  Rx5day    Periodo Estacion  
0   155.1   203.8  1961-1990    Anual  
1   181.0   230.5  1991-2020    Anual  


In [ ]:
indices_estaciones_por_periodo = calcular_indices_etccdi(
    serie_pp,
    periodos=periodos,
    por_estacion=True,
    modo_periodos=True
)

print(indices_estaciones_por_periodo)


   PRCPTOT       SDII  CWD  CDD  R10mm  R20mm  R40mm  R95pTOT  R99pTOT  \
0   8337.2  15.418284    9   33    268    130     47   2168.1    675.5   
1   7828.9  15.818776   10   48    229    118     44   2530.3    845.4   
2   5354.0  11.607912    5   45    183     77     16    582.4      0.0   
3   8316.8  15.049088    8   33    264    148     41   1681.4    688.5   
4   9527.1  16.592105    6   33    285    153     63   3441.7   1007.5   
5   8513.5  16.213052    7   37    250    156     51   1921.7    630.0   
6   6085.4  13.839862    7   39    192    106     33   1165.9    166.5   
7   8030.1  14.015520    6   33    276    137     36   1347.0    179.0   

   Rx1day  Rx5day    Periodo       Estacion  
0   148.6   182.5  1961-1990     Verano_DJF  
1   155.1   196.5  1961-1990      Otoño_MAM  
2    79.0   134.4  1961-1990   Invierno_JJA  
3   133.0   203.8  1961-1990  Primavera_SON  
4   117.0   230.5  1991-2020     Verano_DJF  
5   181.0   196.2  1991-2020      Otoño_MAM  
6    84.5  

In [ ]:
indices_anuales.reset_index(drop=True).to_excel('/content/drive/MyDrive/TP-Mate4/Datos/indices_anuales.xlsx', index=False)
indices_estaciones.reset_index(drop=True).to_excel('/content/drive/MyDrive/TP-Mate4/Datos/indices_estaciones.xlsx', index=False)
indices_anuales_por_periodo.reset_index(drop=True).to_excel('/content/drive/MyDrive/TP-Mate4/Datos/indices_anuales_por_periodo.xlsx', index=False)
indices_estaciones_por_periodo.reset_index(drop=True).to_excel('/content/drive/MyDrive/TP-Mate4/Datos/indices_estaciones_por_periodo.xlsx', index=False)